# Task part 1

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [4]:
# file path
path = r'/Users/elliothaigh/05-2023 Instacart Basket Analysis'

In [5]:
# import data set
df_customer =pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [6]:
df_customer.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [7]:
# rename columns
df_customer = df_customer.rename(columns={'First Name':'first_name', 'Surnam':'last_name', 'Gender':'sex', 'STATE' :'state', 'Age':'age'})

In [8]:
df_customer.shape

(206209, 10)

In [9]:
df_customer.describe()

,user_id,age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [10]:
# check for mixed data columns
for col in df_customer.columns.tolist():
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer[weird]) > 0:
    print (col)

first_name


In [11]:
# change so that first_name column type is string
df_customer['first_name'] = df_customer['first_name'].astype('str')

In [12]:
# check for missing values 
df_customer.isnull().sum()

user_id         0
first_name      0
last_name       0
sex             0
state           0
age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [13]:
# check for duplicates
df_dups = df_customer[df_customer.duplicated()]

In [14]:
df_dups
# there are no duplicates

,user_id,first_name,last_name,sex,state,age,date_joined,n_dependants,fam_status,income


In [15]:
# user_id is str type, we need to change it to an int
type('user_id')

str

In [16]:
df_customer['user_id'] = df_customer['user_id'].astype('int64')

In [17]:
# import latest orders/products dataset
ords_prods_merge =pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_4.8.pkl'))

In [18]:
# change user_id type to int64 to match with df_customer
ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('int64')

In [23]:
# remove existing _merge flag as this was causing an error in our new merge 
del ords_prods_merge['_merge']

In [24]:
# merge
df_merged_large = df_customer.merge(ords_prods_merge, on = 'user_id', indicator = True)

In [25]:
df_merged_large.head()

,user_id,first_name,last_name,sex,state,age,date_joined,n_dependants,fam_status,income,...,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_order,spend_flag,avg_days,frequency_flag,_merge
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Busiest days,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
1,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Regularly busy,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
2,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Busiest days,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
3,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Regularly busy,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
4,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Slowest days,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both


In [27]:
#check value counts for merge coloumn
df_merged_large['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [28]:
df_merged_large.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'customer_ords_prods_merge.pkl'))